In [5]:
import cv2
import numpy as np

In [6]:
# Create a VideoCapture object to capture video from the Raspberry Pi camera
cap = cv2.VideoCapture(0)

# Define the range of colors to detect in the HSV color space
lower_color = np.array([0, 0, 0])
upper_color = np.array([180, 255, 100])

In [7]:
# Loop over each frame in the video stream
while True:
    # Read a frame from the video stream
    ret, frame = cap.read()

    # Resize the frame to a smaller size for faster processing
    resized = cv2.resize(frame, (320, 240))

    # Convert the frame from the BGR color space to the HSV color space
    hsv = cv2.cvtColor(resized, cv2.COLOR_BGR2HSV)

    # Create a binary mask of the pixels in the image that fall within the color range
    mask = cv2.inRange(hsv, lower_color, upper_color)

    # Apply a series of morphological operations to remove noise and fill in small gaps in the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=1)
    mask = cv2.medianBlur(mask, 5)

    # Find contours in the mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours on the original frame
    cv2.drawContours(resized, contours, -1, (0, 255, 0), 3)

    # Detect the lines in the image using the probabilistic Hough transform
    lines = cv2.HoughLinesP(mask, 1, np.pi/180, 50, minLineLength=50, maxLineGap=10)

    # Post-process the lines to remove noise, fill gaps, or smooth the output
    if lines is not None:
        # Convert the lines to a numpy array for easier manipulation
        lines = np.squeeze(lines)

        # Remove horizontal and vertical lines
        lines = [l for l in lines if abs(l[0]-l[2]) > 5 and abs(l[1]-l[3]) > 5]

        # Group lines that are close together
        line_groups = {}
        for l in lines:
            key = round(l[1]/10)*10
            if key not in line_groups:
                line_groups[key] = [l]
            else:
                line_groups[key].append(l)

        # Average the lines in each group to get a single line for each group
        averaged_lines = []
        for key in line_groups:
            group = line_groups[key]
            if len(group) == 1:
                averaged_lines.append(group[0])
            else:
                x1 = np.mean([l[0] for l in group])
                y1 = np.mean([l[1] for l in group])
                x2 = np.mean([l[2] for l in group])
                y2 = np.mean([l[3] for l in group])
                averaged_lines.append([int(x1), int(y1), int(x2), int(y2)])

        # Draw the averaged lines on the original frame
        for line in averaged_lines:
            cv2.line(resized, (line[0], line[1]), (line[2], line[3]), (0, 0, 255), 3)

    # Display the original frame and the mask
    cv2.imshow('Original', resized)
    cv2.imshow('Mask', mask)

    # Exit the program if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object and close all windows
cap.release()
cv2.destroyAllWindows()